In [1]:
# %pip install gurobipy
# %conda install -c fico-xpress xpress -y

In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import xpress as xp
from loguru import logger

np.random.seed(12)

In [3]:
# Let's say we have 5 stations and each station has 2 DSPs
stations = ["BFI5", "DTE6", "DXT7", "DEB8", "DBU9"]
dsps = ["GAZL", "DOBU", "GODL", "AMZL", "AMXL", "DTUY", "FREZ", "CORP", "WADL", "MITF"]

print(f"dsp : {dsps}")
print(f"station : {stations}")

dsp : ['GAZL', 'DOBU', 'GODL', 'AMZL', 'AMXL', 'DTUY', 'FREZ', 'CORP', 'WADL', 'MITF']
station : ['BFI5', 'DTE6', 'DXT7', 'DEB8', 'DBU9']


In [4]:
# station_volume_forecast = dict(
#     zip(stations, np.random.normal(loc=1000, scale=50, size=5).clip(800.0, 1200.0))
# )

# station_spr_forecast = dict(
#     zip(stations, np.random.normal(loc=25.0, scale=10.0, size=5).clip(15.0, 50.0))
# )

# dsp_capacity_forecast = dict(
#     zip(dsps, np.random.normal(loc=25.0, scale=10.0, size=10).clip(15.0, 50.0))
# )

In [5]:
final_data = pd.DataFrame()
for week in range(1, 14):
    station_volume_forecast = dict(
        zip(stations, np.random.normal(loc=1000, scale=50, size=5).clip(800.0, 1200.0))
    )

    station_spr_forecast = dict(
        zip(stations, np.random.normal(loc=25.0, scale=10.0, size=5).clip(15.0, 50.0))
    )

    dsp_capacity_forecast = dict(
        zip(dsps, np.random.normal(loc=25.0, scale=10.0, size=10).clip(15.0, 50.0))
    )
    data = {"week": np.repeat(week, 10), "stations": stations + stations, "dsp": dsps}
    station_dsp = pd.DataFrame(data=data)
    station_dsp = station_dsp.sort_values(by=["stations", "dsp", "week"]).reset_index(
        drop=True
    )
    station_dsp["volume_forecast"] = (
        station_dsp["stations"].map(station_volume_forecast).astype("int32")
    )
    station_dsp["spr_forecast"] = (
        station_dsp["stations"].map(station_spr_forecast).astype("int32")
    )
    station_dsp["routes_demand"] = (
        station_dsp.volume_forecast / station_dsp.spr_forecast
    ).astype("int32")
    station_dsp["dsp_capacity"] = (
        station_dsp["dsp"].map(dsp_capacity_forecast).astype("int32")
    )
    station_dsp["volume_share"] = [0.3, 0.7, 0.6, 0.4, 0.5, 0.5, 0.6, 0.4, 0.8, 0.2]
    station_dsp["dsp_routes_demand"] = (
        station_dsp.routes_demand * station_dsp.volume_share
    )
    station_dsp["is_elligible_target"] = np.where(
        station_dsp.dsp_routes_demand < station_dsp.dsp_capacity, 1, 0
    )
    station_cap = (
        station_dsp.groupby(["stations"])["dsp_capacity"]
        .sum()
        .astype("int32")
        .reset_index()
    )
    station_cap.rename(columns={"dsp_capacity": "station_capacity"}, inplace=True)
    station_dsp = station_dsp.merge(station_cap, on="stations", how="left")
    station_dsp["cap_gap_station"] = (
        station_dsp["station_capacity"] - station_dsp["routes_demand"]
    )
    station_dsp["is_under_solved"] = np.where(station_dsp.cap_gap_station < 0, 1, 0)
    station_dsp["is_elligible_target"] = (
        station_dsp.is_elligible_target - station_dsp.is_under_solved
    ) * station_dsp.is_elligible_target
    final_data = pd.concat([station_dsp, final_data])
    logger.debug(f"code completed for week {week}")

station_dsp = final_data.copy()
station_dsp = station_dsp.sort_values(by=["stations", "dsp", "week"]).reset_index(
    drop=True
)
station_dsp

2024-02-11 15:01:09.189 | DEBUG    | __main__:<module>:54 - code completed for week 1
2024-02-11 15:01:09.198 | DEBUG    | __main__:<module>:54 - code completed for week 2
2024-02-11 15:01:09.205 | DEBUG    | __main__:<module>:54 - code completed for week 3
2024-02-11 15:01:09.212 | DEBUG    | __main__:<module>:54 - code completed for week 4
2024-02-11 15:01:09.219 | DEBUG    | __main__:<module>:54 - code completed for week 5
2024-02-11 15:01:09.225 | DEBUG    | __main__:<module>:54 - code completed for week 6
2024-02-11 15:01:09.230 | DEBUG    | __main__:<module>:54 - code completed for week 7
2024-02-11 15:01:09.236 | DEBUG    | __main__:<module>:54 - code completed for week 8
2024-02-11 15:01:09.241 | DEBUG    | __main__:<module>:54 - code completed for week 9
2024-02-11 15:01:09.245 | DEBUG    | __main__:<module>:54 - code completed for week 10
2024-02-11 15:01:09.251 | DEBUG    | __main__:<module>:54 - code completed for week 11
2024-02-11 15:01:09.255 | DEBUG    | __main__:<modul

,week,stations,dsp,volume_forecast,spr_forecast,routes_demand,dsp_capacity,volume_share,dsp_routes_demand,is_elligible_target,station_capacity,cap_gap_station,is_under_solved
0,1,BFI5,DTUY,1023,15,68,23,0.3,20.4,0,42,-26,1
1,2,BFI5,DTUY,1025,15,68,25,0.3,20.4,0,44,-24,1
2,3,BFI5,DTUY,971,19,51,24,0.3,15.3,1,51,0,0
3,4,BFI5,DTUY,1047,27,38,41,0.3,11.4,1,64,26,0
4,5,BFI5,DTUY,933,15,62,15,0.3,18.6,0,36,-26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,9,DXT7,GODL,1029,18,57,22,0.2,11.4,0,43,-14,1
126,10,DXT7,GODL,1038,35,29,15,0.2,5.8,1,43,14,0
127,11,DXT7,GODL,1039,31,33,32,0.2,6.6,1,62,29,0
128,12,DXT7,GODL,1019,19,53,36,0.2,10.6,1,72,19,0


In [6]:
undersolved_stations = list(
    station_dsp[station_dsp.is_under_solved == 1].stations.unique()
)
eligible_targets = list(station_dsp[station_dsp.is_elligible_target == 1].dsp.unique())
roles = ["popup", "pinnacle", "transfer", "newdsp"]
launch_time = range(1, 14)

# dsp_roles = []
# for d in eligible_targets:
#     for role in roles:
#         if dsp_roles:
#             dsp_roles.append((d, role))
#         else:
#             dsp_roles = [(d, role)]

# dsp_role_stations = []
# for d in eligible_targets:
#     for r in roles:
#         for s in undersolved_stations:
#             if dsp_role_stations:
#                 dsp_role_stations.append((d, r, s))
#             else:
#                 dsp_role_stations = [(d, r, s)]

dsp_role_station_launches = []
for d in eligible_targets:
    for r in roles:
        for s in undersolved_stations:
            for t in launch_time:
                if dsp_role_station_launches:
                    dsp_role_station_launches.append((d, r, s, t))
                else:
                    dsp_role_station_launches = [(d, r, s, t)]

# print(len(dsp_role_stations))
print(len(dsp_role_station_launches))

2600


In [7]:
deployment_fixed_cost = (
    np.random.normal(loc=3.0, scale=1.0, size=len(dsp_role_station_launches)).clip(
        1.0, 5.0
    )
).tolist()

print(len(deployment_fixed_cost))

2600


In [ ]:
combinations, deployment_fixed_cost = gp.multidict(
    dict(
        zip(
            dsp_role_station_launches,
            np.array(deployment_fixed_cost),
        )
    )
)

# dsp_role_station_combo, capacity_projection = gp.multidict(
#     dict(
#         zip(
#             dsp_role_stations,
#             np.random.normal(loc=7.0, scale=2.5, size=len(dsp_role_stations)).clip(
#                 5.0, 10.0
#             ),
#         )
#     )
# )

combinations, temporal_capacity_projections = gp.multidict(
    dict(
        zip(
            dsp_role_station_launches,
            np.random.normal(
                loc=20.0, scale=10.0, size=len(dsp_role_station_launches)
            ).clip(5.0, 35.0),
        )
    )
)

print(len(dsp_role_station_launches))

In [ ]:
combinations[0:5]

In [ ]:
for i in (x.sum(D, "*", "*", T) <= 1 for D in eligible_targets for T in launch_time):
    print(i)
    break

# Gurobi Model

In [ ]:
# Initiate the model
m = gp.Model("Detor")

# Define Variables
x = m.addVars(
    combinations,
    vtype=GRB.BINARY,
    name="target_deployment",
)

# Define Constraints
# A DSP target combination can be assigned to only one station
station_assignment_constraint = m.addConstrs(
    (x.sum(D, "*", "*", T) <= 1 for D in eligible_targets for T in launch_time),
    name="target_station",
)

# # Sum of target capacity at a station shouldn't exceed its capacity gap
# station_capacity_gap_constraint = m.addConstrs(
#     (x.prod(capacity_projections, S) == [26, 27] for S in undersolved_stations),
#     name="cap_gap_station",
# )

# A DSP can assume only one role at a station
# dsp_role_constraint = m.addConstrs(
#     (y.sum(T, "*") <= 1 for T in eligible_targets), name="dsp_role"
# )


# Define Objective Function
m.setObjective(
    5 * 53 - 2 * x.prod(temporal_capacity_projections) + x.prod(deployment_fixed_cost),
    GRB.MINIMIZE,
)


# Run Optimization
m.optimize()

In [ ]:
m.write("Detor_single_week.lp")

# Xpress Model

In [14]:
from xpress import *

In [15]:
detor = xp.problem(name="DSP Target Deployment Optimizer", sense=xp.minimize)

Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Users/divye/opt/anaconda3/envs/ml39/lib/python3.9/site-packages/xpress/license/community-xpauth.xpr')


In [19]:
x = xp.vars(dsp_role_station_launches, vartype=xp.binary, name="target_deployment")
detor.addVariable(x)
type(x)

?120 Error: Problem has too many rows and columns. The maximum is 5000


SolverError: ?120 Error: Problem has too many rows and columns. The maximum is 5000

In [12]:
x.sum

dict_keys([('DTUY', 'popup', 'BFI5', 1), ('DTUY', 'popup', 'BFI5', 2), ('DTUY', 'popup', 'BFI5', 3), ('DTUY', 'popup', 'BFI5', 4), ('DTUY', 'popup', 'BFI5', 5), ('DTUY', 'popup', 'BFI5', 6), ('DTUY', 'popup', 'BFI5', 7), ('DTUY', 'popup', 'BFI5', 8), ('DTUY', 'popup', 'BFI5', 9), ('DTUY', 'popup', 'BFI5', 10), ('DTUY', 'popup', 'BFI5', 11), ('DTUY', 'popup', 'BFI5', 12), ('DTUY', 'popup', 'BFI5', 13), ('DTUY', 'popup', 'DBU9', 1), ('DTUY', 'popup', 'DBU9', 2), ('DTUY', 'popup', 'DBU9', 3), ('DTUY', 'popup', 'DBU9', 4), ('DTUY', 'popup', 'DBU9', 5), ('DTUY', 'popup', 'DBU9', 6), ('DTUY', 'popup', 'DBU9', 7), ('DTUY', 'popup', 'DBU9', 8), ('DTUY', 'popup', 'DBU9', 9), ('DTUY', 'popup', 'DBU9', 10), ('DTUY', 'popup', 'DBU9', 11), ('DTUY', 'popup', 'DBU9', 12), ('DTUY', 'popup', 'DBU9', 13), ('DTUY', 'popup', 'DEB8', 1), ('DTUY', 'popup', 'DEB8', 2), ('DTUY', 'popup', 'DEB8', 3), ('DTUY', 'popup', 'DEB8', 4), ('DTUY', 'popup', 'DEB8', 5), ('DTUY', 'popup', 'DEB8', 6), ('DTUY', 'popup', 'DE

In [ ]:
for v in m.getVars():
    if v.x > 0.0:
        print(v.varName, v.x)

In [ ]:
x.prod(capacity_projection).getValue(), x.prod(deployment_fixed_cost).getValue()

In [ ]:
package_cost = np.sum(
    np.abs(
        station_dsp.cap_gap_station
        * station_dsp.spr_forecast
        * station_dsp.is_under_solved
    )
    / 2
)
excess_capacity_cost = 0.1 * np.sum(
    (
        station_dsp[station_dsp.is_under_solved == 0]["cap_gap_station"]
        * station_dsp.spr_forecast
    )
    / 2
)
print(f"current cost of system = {package_cost} + {excess_capacity_cost}")

In [ ]:
elligible_dsps = station_dsp[station_dsp.is_elligible_target == 1].dsp.unique()
print(elligible_dsps)

popups_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=15, scale=10.0, size=len(elligible_dsps)).clip(5.0, 30.0),
    )
)

pinnacle_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=20, scale=10.0, size=len(elligible_dsps)).clip(5.0, 30.0),
    )
)

transfer_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=30, scale=10.0, size=len(elligible_dsps)).clip(15.0, 50.0),
    )
)

new_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=10, scale=10.0, size=len(elligible_dsps)).clip(0.0, 20.0),
    )
)

data = {"dsp": elligible_dsps}
capacity_add = pd.DataFrame(data)

capacity_add["popup"] = capacity_add["dsp"].map(popups_forecast)
capacity_add["pinnacle"] = capacity_add["dsp"].map(pinnacle_forecast)
capacity_add["transfer"] = capacity_add["dsp"].map(transfer_forecast)
capacity_add["new"] = capacity_add["dsp"].map(new_forecast)
capacity_add

In [ ]:
n_elligible_dsps = station_dsp[station_dsp.is_elligible_target == 0].dsp.unique()
print(n_elligible_dsps)

popups_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=3.0, scale=1.0, size=len(n_elligible_dsps)).clip(0.0, 5.0),
    )
)

pinnacle_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=3.0, scale=1.0, size=len(elligible_dsps)).clip(0.0, 5.0),
    )
)

data = {"dsp": elligible_dsps}
capacity_sub = pd.DataFrame(data)

capacity_sub["popup"] = capacity_sub["dsp"].map(popups_forecast)
capacity_sub["pinnacle"] = capacity_sub["dsp"].map(pinnacle_forecast)
capacity_sub["transfer"] = capacity_sub["dsp"].map(transfer_forecast)
capacity_sub["new"] = 0
capacity_sub

In [ ]:
scenario = np.array(capacity_add[["popup", "pinnacle", "transfer", "new"]]).flatten()
oversolved = pd.DataFrame({"station": oversolved_stations})

In [ ]:
x = np.array(np.repeat([1, 0, 0, 0], 5)).reshape(1, 20)
x

In [ ]:
A_ub = np.sum(x, axis=1).reshape(5, 1)
A_ub

In [ ]:
A = np.repeat(1, 5).reshape(5, 1)
A

In [ ]:
map_x = np.repeat(x.flatten().reshape(1, 20), 2).reshape(len(undersolved_stations), 20)
map_x

In [ ]:
b_ub = np.sum(map_x, axis=0)
b_ub

In [ ]:
B = np.repeat(1, 20).reshape(1, 20)
B

In [ ]:
bounds = list(zip(np.repeat(0, 20), np.repeat(1, 20)))
bounds

In [ ]:
# c = -np.array(list(price.values()))
# c

In [ ]:
# A_ub = np.array([np.array(list(w.values())), np.array(list(v.values()))])

# A_ub

In [ ]:
b_ub = np.array([kw, kv])
b_ub

In [ ]:
bounds = [
    (0, 1),
] * 10
bounds

In [ ]:
from scipy.optimize import linprog

# Obtain solution
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

print(sol)

In [ ]:
integrality_vector = np.full(c.shape[0], 1)
print(integrality_vector)
sol_int = linprog(
    c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=integrality_vector
)
print(sol_int)